# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [18]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
# importing the libraries
import pandas as pd
import csv
import sqlite3

# Reading the dataset
for chunk in  pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000):
  print(chunk.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       5000 non-null   object 
 1   company_name            5000 non-null   object 
 2   company_category_code   4948 non-null   object 
 3   company_country_code    5000 non-null   object 
 4   company_state_code      4947 non-null   object 
 5   company_region          5000 non-null   object 
 6   company_city            4936 non-null   object 
 7   investor_permalink      5000 non-null   object 
 8   investor_name           5000 non-null   object 
 9   investor_category_code  2443 non-null   object 
 10  investor_country_code   4222 non-null   object 
 11  investor_state_code     3629 non-null   object 
 12  investor_region         5000 non-null   object 
 13  investor_city           4100 non-null   object 
 14  funding_round_type      5000 non-null   

In [3]:
# Counting missing values in the chunk
print(" \nTotal count of NaN(missing values) at each column in crunchbase-investments.csv : \n\n", chunk.isnull().sum())

 
Total count of NaN(missing values) at each column in crunchbase-investments.csv : 

 company_permalink            0
company_name                 0
company_category_code       10
company_country_code         0
company_state_code          29
company_region               0
company_city                45
investor_permalink           0
investor_name                0
investor_category_code    2870
investor_country_code     2870
investor_state_code       2870
investor_region              0
investor_city             2870
funding_round_type           0
funded_at                    0
funded_month                 0
funded_quarter               0
funded_year                  0
raised_amount_usd          290
dtype: int64


In [4]:
# Finding the memory usge of each column in bytes
chunk.memory_usage(deep=True)

Index                        132
company_permalink         215412
company_name              189709
company_category_code     184581
company_country_code      172200
company_state_code        168547
company_region            184997
company_city              191495
investor_permalink        222759
investor_name             199818
investor_category_code     22960
investor_country_code      22960
investor_state_code        22960
investor_region           183680
investor_city              22960
funding_round_type        182367
funded_at                 192290
funded_month              183680
funded_quarter            183680
funded_year                22960
raised_amount_usd          22960
dtype: int64

In [5]:
#Calculating the total memory usage of each columns in MB's
for chunk in  pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000):
  mem = (chunk.memory_usage(deep=True).sum())*0.000001
  print(mem,"- megabytes")

5.850257999999999 - megabytes
5.796772 - megabytes
5.803921 - megabytes
5.796730999999999 - megabytes
5.792688 - megabytes
5.823191 - megabytes
5.800132 - megabytes
5.777296 - megabytes
5.658243 - megabytes
4.864866 - megabytes
2.793107 - megabytes


In [6]:
#placing the chunk column memory totals to a list
total_chunk_memory_list = []
for chunk in  pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000):
  mem = (chunk.memory_usage(deep=True).sum())*0.000001
  total_chunk_memory_list.append(mem)
print("Chunk Memory List(in mbs):",total_chunk_memory_list)

Chunk Memory List(in mbs): [5.850257999999999, 5.796772, 5.803921, 5.796730999999999, 5.792688, 5.823191, 5.800132, 5.777296, 5.658243, 4.864866, 2.793107]


In [7]:
#getting the total chunk memory
total_chunk_memory = sum(total_chunk_memory_list)
print("TOTAL MEMORY(in mbs):",total_chunk_memory)

TOTAL MEMORY(in mbs): 59.75720499999999


In [8]:
# dropping columns that have missing data.
crunch = pd.read_csv("crunchbase-investments.csv", encoding='latin1')
crunch.drop(['investor_category_code', 'investor_country_code','investor_state_code', 'investor_city','raised_amount_usd','company_city','company_state_code','company_category_code',], axis = 1, inplace = True)
for col in crunch.columns:
    print(col)

company_permalink
company_name
company_country_code
company_region
investor_permalink
investor_name
investor_region
funding_round_type
funded_at
funded_month
funded_quarter
funded_year


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [9]:
# Your code goes here
#Getting to know the datatype of the columns and memry usage.
print(crunch.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52870 entries, 0 to 52869
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   company_permalink     52869 non-null  object 
 1   company_name          52869 non-null  object 
 2   company_country_code  52869 non-null  object 
 3   company_region        52869 non-null  object 
 4   investor_permalink    52868 non-null  object 
 5   investor_name         52868 non-null  object 
 6   investor_region       52868 non-null  object 
 7   funding_round_type    52867 non-null  object 
 8   funded_at             52867 non-null  object 
 9   funded_month          52867 non-null  object 
 10  funded_quarter        52867 non-null  object 
 11  funded_year           52867 non-null  float64
dtypes: float64(1), object(11)
memory usage: 38.8 MB
None


In [10]:
crunch.dtypes

company_permalink        object
company_name             object
company_country_code     object
company_region           object
investor_permalink       object
investor_name            object
investor_region          object
funding_round_type       object
funded_at                object
funded_month             object
funded_quarter           object
funded_year             float64
dtype: object

In [11]:
# Getting the columns with numerics
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64','complex']

crunch_numeric = crunch.select_dtypes(include=numerics)
print("Columns that have numeric types:",crunch_numeric.columns.tolist())

Columns that have numeric types: ['funded_year']


In [12]:
# Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
# See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
# Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.

crunch = pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000)
for unique in crunch:
  print(unique.value_counts())

company_permalink                           company_name                       company_category_code  company_country_code  company_state_code  company_region  company_city   investor_permalink                 investor_name             investor_category_code  investor_country_code  investor_state_code  investor_region  investor_city  funding_round_type  funded_at   funded_month  funded_quarter  funded_year  raised_amount_usd
/company/massachusetts-clean-energy-center  Massachusetts Clean Energy Center  cleantech              USA                   MA                  Boston          Boston         /company/north-shore-innoventures  North Shore InnoVentures  biotech                 USA                    MA                   Boston           Beverly        angel               2013-09-05  2013-09       2013-Q3         2013         300000.0             2
/company/1000memories                       1000memories                       web                    USA                   CA           

In [ ]:
# from the above data l have not found any column l can change to numeric.


## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [29]:
# Your code goes here

conn = sqlite3.connect('crunch')
crunch = pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000)
for chunk in crunch:
    chunk.to_sql("funding", conn, if_exists='append', index=False)
    df = pd.read_sql('PRAGMA table_info(funding);', conn)
print(df)

    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     TEXT        0       None   0
10   10   investor_country_code     TEXT        0       None   0
11   11     investor_state_code     TEXT        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     TEXT        0       None   0
14   14      funding_roun

## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [31]:
# Your code goes here
# Read sqlite query results into a pandas DataFrame
crunch = pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000)
con = sqlite3.connect("crunch")

df = pd.read_sql_query("SELECT * from funding", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

     company_permalink company_name company_category_code  \
0    /company/advercar     AdverCar           advertising   
1  /company/launchgram   LaunchGram                  news   
2        /company/utap         uTaP             messaging   
3    /company/zoopshop     ZoopShop              software   
4    /company/efuneral     eFuneral                   web   

  company_country_code company_state_code         company_region  \
0                  USA                 CA                 SF Bay   
1                  USA                 CA                 SF Bay   
2                  USA               None  United States - Other   
3                  USA                 OH               Columbus   
4                  USA                 OH              Cleveland   

    company_city          investor_permalink      investor_name  \
0  San Francisco  /company/1-800-flowers-com  1-800-FLOWERS.COM   
1  Mountain View        /company/10xelerator        10Xelerator   
2           None       

In [61]:
# What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
raised = 'select raised_amount_usd, count(*) as counts from funding group by raised_amount_usd  order by counts desc;'
percent = pd.read_sql(raised, conn)
print(percent[:10])

   raised_amount_usd  counts
0                NaN   10797
1         10000000.0    6201
2          1000000.0    5280
3          5000000.0    4767
4         15000000.0    4260
5         20000000.0    3936
6          2000000.0    3759
7          1500000.0    3741
8          3000000.0    3612
9          6000000.0    3444


In [40]:
raised = 'select raised_amount_usd, count(*) as counts from funding group by raised_amount_usd  order by counts asc;'
percent = pd.read_sql(raised, conn)
print(percent[:10])

   raised_amount_usd  counts
0             2100.0       3
1             5500.0       3
2             5600.0       3
3             6000.0       3
4             8300.0       3
5            11600.0       3
6            11800.0       3
7            13200.0       3
8            15282.0       3
9            16500.0       3


In [62]:
# Which category of company attracted the most investments?
raised = 'select company_category_code, count(*) as counts from funding group by raised_amount_usd  order by counts desc;'
percent = pd.read_sql(raised, conn)
print(percent[:1])

  company_category_code  counts
0             ecommerce   10797


In [45]:
# Which investor contributed the most money (across all startups)?
raised = 'select investor_name, count(*) as counts from funding group by raised_amount_usd and company_name order by counts desc;'
percent = pd.read_sql(raised, conn)
print(percent[:1])

  investor_name  counts
0  Zorik Gordon  157584


In [68]:
# Which investors contributed the most money per startup?
raised = 'select investor_name,raised_amount_usd , count(*) as counts from funding group by raised_amount_usd order by raised_amount_usd  desc;'
percent = pd.read_sql(raised, conn)
print(percent[:1])

  investor_name  raised_amount_usd  counts
0   Time Warner       3.200000e+09      15


In [73]:
# Which funding round was the most popular? 
raised = 'select funding_round_type , count(*) as counts from funding group by funding_round_type order by counts desc;'
percent = pd.read_sql(raised, conn)
print(percent[:1])

  funding_round_type  counts
0           series-a   41814


In [76]:
# Which was the least popular?
raised = 'select funding_round_type , count(*) as counts from funding group by funding_round_type order by counts asc;'
percent = pd.read_sql(raised, conn)
print(percent[:1])

  funding_round_type  counts
0               None       9
